# PhosphoELM Data Formating

This file takes data regarding kinase-protein interactions from the PhosphoELM database and converts the data into the .gmt format. The data was retrieved from the PhosphoELM database on Wed, Jun 7 2017 16:27:31. This data will be added to enhance the KEA2 database and will be suitably formatted for use by ENRICHR and X2K.

## Import packages necessary for following program

In [148]:
%run /home/maayanlab/Projects/Scripts/init.ipy

## Create a dataframe from a file containing PhosphoELM data

In [149]:
#read data from excel file into dataframe 'phospho_df'
phospho_df = pd.read_excel('~/Desktop/phosphoELM_all_2015-04.xlsm')
phospho_df.head()

acc                                           sequence  position code  \
0  O08539  MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...       304    S   
1  O08539  MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...       304    S   
2  O08539  MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...       304    S   
3  O08539  MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...       296    S   
4  O08539  MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...       296    S   

      pmids kinases source       species                     entry_date  
0  17114649     NaN    HTP  Mus musculus  2005-03-14 12:16:11.108314+01  
1  17242355     NaN    HTP  Mus musculus  2005-03-14 12:16:11.108314+01  
2  15345747     NaN    HTP  Mus musculus  2005-03-14 12:16:11.108314+01  
3  17114649     NaN    HTP  Mus musculus  2007-07-13 15:17:45.666219+02  
4  17242355     NaN    HTP  Mus musculus  2007-07-13 15:17:45.666219+02

## Filter by Organism

In [150]:
# define a list of selected organisms
organisms = ['Mus musculus', 'Homo sapiens']

# get indices of rows whose species is in the selected organisms
indices = [index for index, rowData in phospho_df.iterrows() if rowData['species'] in organisms]

# filter
phospho_df_filter = phospho_df.loc[indices, ['acc', 'kinases', 'species']].dropna()
phospho_df_filter.head()

acc    kinases       species
9   O08605       PAK2  Mus musculus
10  O08605       PAK2  Mus musculus
14  O14543        Lck  Homo sapiens
16  O14543        Lck  Homo sapiens
18  O14746  PKB_group  Homo sapiens

## Convert UniProt IDs to Gene Symbols

In [151]:
# Use uniprot_to_symbol function from Scripts.py to convert
phospho_df_filter['target_symbol'] = Scripts.uniprot_to_symbol(phospho_df_filter['acc'].tolist())
phospho_df_filter.head()

acc    kinases       species target_symbol
9   O08605       PAK2  Mus musculus         Mknk1
10  O08605       PAK2  Mus musculus         Mknk1
14  O14543        Lck  Homo sapiens         SOCS3
16  O14543        Lck  Homo sapiens         SOCS3
18  O14746  PKB_group  Homo sapiens          TERT

## Create a new column combining kinases and organism

In [152]:
# Combine 'kinases' and 'species' into one column 'kinase_organism'
phospho_df_filter['kinase_organism'] = ['_'.join([kinase, species]) for kinase, species in phospho_df_filter[['kinases', 'species']].as_matrix()]
phospho_df_filter.head()

acc    kinases       species target_symbol         kinase_organism
9   O08605       PAK2  Mus musculus         Mknk1       PAK2_Mus musculus
10  O08605       PAK2  Mus musculus         Mknk1       PAK2_Mus musculus
14  O14543        Lck  Homo sapiens         SOCS3        Lck_Homo sapiens
16  O14543        Lck  Homo sapiens         SOCS3        Lck_Homo sapiens
18  O14746  PKB_group  Homo sapiens          TERT  PKB_group_Homo sapiens

## Perform preliminary data processing 

Select columns necessary for .gmt format and filter into new dataframe 'df'
df = phospho_df[['acc', 'kinases', 'species']]We must drop duplicates and NaNs, as well as select only the columns necessary for the .gmt file format (the protein ids and kinase gene symbols). The species column is also selected for future filtering of data by desired species.

In [153]:
#select columns necessary for .gmt format and filter into new dataframe 'df'
df = phospho_df_filter[['target_symbol', 'kinase_organism']]

#drop duplicate rows in the dataframe
df.drop_duplicates(inplace = True)

#drop all rows with an 'NaN' value for the kinases
df.dropna(axis = 0, inplace = True)

#Visualize data
df.head()

/home/maayanlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/maayanlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


target_symbol         kinase_organism
9          Mknk1       PAK2_Mus musculus
14         SOCS3        Lck_Homo sapiens
18          TERT  PKB_group_Homo sapiens
21          TERT        SRC_Homo sapiens
27         IKBKB  IKK_group_Homo sapiens

## Set Index to 'Kinase_Organism' and Aggregate Kinase Targets

In [154]:
df.set_index('kinase_organism')

#Group kinases in dataframe 'kin'
#Aggregate data in 'kin' according to kinase groups
kin = df.groupby('kinase_organism').agg(lambda x: tuple(x))

#Create a new column 'PhosphoELM' as description of data
kin.insert(0, 'Description', 'PhosphoELM')

#Visualize Data
kin.head()

Description  \
kinase_organism                       
AAK1_Homo sapiens        PhosphoELM   
ALK_Homo sapiens         PhosphoELM   
AMPK_group_Homo sapiens  PhosphoELM   
AMPK_group_Mus musculus  PhosphoELM   
ATM_Homo sapiens         PhosphoELM   

                                                             target_symbol  
kinase_organism                                                             
AAK1_Homo sapiens                                     (NUMB, AP1M1, AP2M1)  
ALK_Homo sapiens                                      (STAT3, ALK, ZC3HC1)  
AMPK_group_Homo sapiens  (IRS1, KPNA2, EP300, EEF2K, PFKFB2, HNF4A, PRK...  
AMPK_group_Mus musculus                                       (Irs1, Mtor)  
ATM_Homo sapiens         (CHEK2, ABL1, TP53, ATF2, RPA2, CREB1, BRCA1, ...

## Create column representing count of Protein targets for each kinase

In [159]:
# Create column representing counts of protein targets per kinase
kin['kinase_targets_num'] = [len(lst) for kinase, lst in kin['target_symbol'].iteritems()]
kin.sort_values(by = ['kinase_targets_num'], ascending= False, inplace=True)
kin.head()

Description  \
kinase_organism                      
PKA_group_Homo sapiens  PhosphoELM   
CK2_group_Homo sapiens  PhosphoELM   
PKC_group_Homo sapiens  PhosphoELM   
CDK1_Homo sapiens       PhosphoELM   
PKC_alpha_Homo sapiens  PhosphoELM   

                                                            target_symbol  \
kinase_organism                                                             
PKA_group_Homo sapiens  (HIST1H2BI, FGA, ESR1, RAF1, ANXA1, TH, ADRB2,...   
CK2_group_Homo sapiens  (C1R, JUN, SSB, PGR, CDK1, HSP90AA1, HNRNPC, S...   
PKC_group_Homo sapiens  (EGFR, IL2RA, LMNA, FGA, TFRC, RAF1, ANXA1, EE...   
CDK1_Homo sapiens       (BIRC5, EGFR, TK1, KRT8, NPM1, PPP1CA, VIM, TO...   
PKC_alpha_Homo sapiens  (EGFR, TFRC, ANXA1, INSR, ANXA2, PLEK, SRF, CF...   

                        kinase_targets_num  
kinase_organism                             
PKA_group_Homo sapiens                 139  
CK2_group_Homo sapiens                  84  
PKC_group_Homo sapiens                  81  
CDK1_Homo sapiens                       67  
PKC_alpha_Homo sapiens                  67

## Create Dictionary of Tab-Separated Rows of the Dataframe

In [ ]:
#Reset index of the dataframe
kin.reset_index(inplace = True)

#create column 'acc_merged' in which all 'acc' elements are joined by a \t symbol
kin['target_symbol_merged'] = ['\t'.join(x) for x in kin['target_symbol']]

#drop the now-unneccesary column 'target_symbol'
kin.drop('target_symbol', axis=1, inplace = True)

#Create dictionary 'PhosphoELM' with index numbers as keys
PhosphoELM_num = dict([(key, '') for key in kin.index])

# loop through rows with iterrows()
for index, rowData in kin.iterrows():
    line = ('\t'.join(rowData))
    PhosphoELM_num[index] = line

## Write Info from Dictionary into a .GMT file

In [ ]:
#Transfer tab-separated info into a new txt file
with open('PhosphoELM.gmt', 'w') as openfile:
    for index in PhosphoELM_num:
        openfile.write(str(PhosphoELM_num[index]) + '\n')

## Test: Reading in the Newly-Created .GMT File

In [160]:
lst = ['protein']
df2 = pd.read_table('PhosphoELM.gmt', delimiter = '\t', names = lst*141)
df2 = df2.replace(np.nan, '', regex=True)
df2.rename(columns = {'protein': 'kinase'}, inplace = True)
df2.rename(columns = {'protein.1': 'Description'}, inplace = True)
df2


kinase Description protein.2 protein.3  \
0                  AAK1_Homo sapiens  PhosphoELM      NUMB     AP1M1   
1                   ALK_Homo sapiens  PhosphoELM     STAT3       ALK   
2            AMPK_group_Homo sapiens  PhosphoELM      IRS1     KPNA2   
3            AMPK_group_Mus musculus  PhosphoELM      Irs1      Mtor   
4                   ATM_Homo sapiens  PhosphoELM     CHEK2      ABL1   
5                   ATM_Mus musculus  PhosphoELM     H2afx             
6                   ATR_Homo sapiens  PhosphoELM     CREB1     BRCA1   
7                  Abl2_Homo sapiens  PhosphoELM       CAT     SIVA1   
8                   Abl_Homo sapiens  PhosphoELM    PLSCR1      TP73   
9                   Abl_Mus musculus  PhosphoELM      Abl1      Cav1   
10             Aurora A_Homo sapiens  PhosphoELM      TP53  HIST1H3J   
11             Aurora A_Mus musculus  PhosphoELM     H3f3b     Cpeb1   
12             Aurora B_Homo sapiens  PhosphoELM     BIRC5       VIM   
13             Aurora B_Mus musculus  PhosphoELM       Vim     H3f3b   
14                  Axl_Homo sapiens  PhosphoELM       AXL             
15                  BLK_Homo sapiens  PhosphoELM    FCGR2A    FCGR2B   
16               BMPR1B_Homo sapiens  PhosphoELM     SMAD2     SMAD1   
17                  BMX_Homo sapiens  PhosphoELM     RUFY1             
18                BRSK1_Homo sapiens  PhosphoELM    CDC25B    CDC25C   
19                  BTK_Homo sapiens  PhosphoELM       WAS       BTK   
20                  Brk_Homo sapiens  PhosphoELM       PXN      PTK6   
21                CCDPK_Homo sapiens  PhosphoELM       PLN       SPR   
22                CDK11_Homo sapiens  PhosphoELM     EIF3F             
23                 CDK1_Homo sapiens  PhosphoELM     BIRC5      EGFR   
24                 CDK1_Mus musculus  PhosphoELM    Nup210      Pak1   
25                 CDK2_Homo sapiens  PhosphoELM      TP73       TK1   
26                 CDK2_Mus musculus  PhosphoELM     Mybl2     Myod1   
27                 CDK3_Homo sapiens  PhosphoELM      ATF1             
28                 CDK4_Homo sapiens  PhosphoELM      RBL1      RBL2   
29                 CDK4_Mus musculus  PhosphoELM     Cebpa             
..                               ...         ...       ...       ...   
341                TIE1_Mus musculus  PhosphoELM      Tie1             
342                TIE2_Homo sapiens  PhosphoELM       TEK             
343                TIE2_Mus musculus  PhosphoELM       Tek             
344                TLK1_Homo sapiens  PhosphoELM     RAD9A             
345                TNK2_Homo sapiens  PhosphoELM      TNK2             
346                TRKA_Homo sapiens  PhosphoELM     NTRK1             
347                TRKB_Mus musculus  PhosphoELM     Ntrk2      Frs3   
348               TSSK3_Mus musculus  PhosphoELM     Tssk3             
349                 TTK_Homo sapiens  PhosphoELM     CHEK2      ABL1   
350                 TXK_Homo sapiens  PhosphoELM      LCP2     CTLA4   
351        Titin kinase_Homo sapiens  PhosphoELM      TCAP             
352        Titin kinase_Mus musculus  PhosphoELM      Tcap             
353  Tropomyosin kinase_Mus musculus  PhosphoELM      Tpm1             
354                Tyk2_Homo sapiens  PhosphoELM    IFNAR1      TYK2   
355                Tyk2_Mus musculus  PhosphoELM   Il13ra1             
356                VRK1_Homo sapiens  PhosphoELM      TP53       JUN   
357                WEE1_Homo sapiens  PhosphoELM      CDK1             
358                WNK1_Homo sapiens  PhosphoELM     OXSR1     STK39   
359                WNK1_Mus musculus  PhosphoELM      Syt2             
360                 Yes_Homo sapiens  PhosphoELM       CBL      YES1   
361                 Yes_Mus musculus  PhosphoELM      Yes1             
362               ZAP70_Homo sapiens  PhosphoELM      SHC1      LCP2   
363               ZAP70_Mus musculus  PhosphoELM       Lat             
364                mTOR_Homo sapiens  PhosphoELM   RPS6KB1      MTOR  

In [ ]:
##Display figures regarding the data
#look into plotly

# This is a title
## This is a subtitle

hello you can write in **bold**, *italic*

lists:
- asd
- asd
- asd


numbered lists:
1. asd
2. 234
3. 46